In [1]:
import Ipynb_importer
from a_basic_quant import *
from b_model import *
from c_train_and_test import *
from d_post_training_quantize import *

importing Jupyter notebook from a_basic_quant.ipynb
importing Jupyter notebook from b_model.ipynb
importing Jupyter notebook from c_train_and_test.ipynb
importing Jupyter notebook from d_post_training_quantize.ipynb


## 1、理解量化过程·教程
### 1.1 获得一个输入 torch.tensor

In [2]:
x = torch.randn(2,3) *100
x

tensor([[-90.1498,  -4.1381, -35.8254],
        [102.5455, 132.4951, -70.6038]])

### 1.2 获得 x 的最大值和最小值，确定上下限

In [3]:
max_val = x.max()
min_val = x.min()
print(f"max_val: {max_val} \nmin_val:{min_val}")

max_val: 132.49508666992188 
min_val:-90.14982604980469


### 1.3 计算缩放因子 scare, 零点 zero_point

In [4]:
scale, zero_point = calcu_scale_and_zeropoint(min_val, max_val)
print(f"scale: {scale} \nzero_point: {zero_point}")

scale: 0.8731173276901245 
zero_point: 103.0


### 1.4 进行量化

In [5]:
q_x = quantize_tensor(x, scale, zero_point)
q_x

tensor([[  0.0000,  98.2606,  61.9684],
        [220.4475, 254.7495,  22.1359]])

### 1.5 反量化

In [6]:
x_q_x = dequantize_tensor(q_x, scale, zero_point)

print(f"原始值：\n{x} \n量化反量化：\n{x_q_x}")

原始值：
tensor([[-90.1498,  -4.1381, -35.8254],
        [102.5455, 132.4951, -70.6038]]) 
量化反量化：
tensor([[-89.9311,  -4.1381, -35.8254],
        [102.5455, 132.4951, -70.6038]])


## 2、深入理解卷积中量化的细节

### 2.1 定义一个输入并且输出指定范围数据

In [7]:
a = torch.randn((64,1,28,28))
a[0][0][0][:5]

tensor([ 2.1448,  0.4215, -0.3201, -1.2773,  0.8699])

### 2.2 定义一个卷积

In [8]:
conv = torch.nn.Conv2d(1, 40, 3, 1)

### 2.3 进行卷积运算并且输出指定范围数据

In [9]:
res = conv(a)
res[0][0][0][:5]

tensor([ 0.8252,  0.2801,  0.6957, -1.0605,  1.0631], grad_fn=<SliceBackward>)

> 上面一步的目的是模拟正常运算，此时网络中的权重和偏置已存在

### 2.4 量化
#### 1、 进行对输入的量化

In [10]:
# 1
min_a, max_a = calcu_max_and_min(a, None, None, False)
# 2
scale_a, zero_point_a = calcu_scale_and_zeropoint(min_a, max_a, 8, False)
# 3
q_a = quantize_tensor(a, scale_a, zero_point_a).int()
# 4
dq_a = dequantize_tensor(q_a, scale_a, zero_point_a)

print("原始：", a[0][0][0][:5])
print("量化：", q_a[0][0][0][:5])
print("反量化：", dq_a[0][0][0][:5])

原始： tensor([ 2.1448,  0.4215, -0.3201, -1.2773,  0.8699])
量化： tensor([198, 145, 123,  93, 159], dtype=torch.int32)
反量化： tensor([ 2.1136,  0.3902, -0.3252, -1.3007,  0.8455])


#### 2、 进行对权重的量化
两种量化方式，逐层量化、逐通道量化

这里展示的是逐通道量化

In [11]:
# 1、获取权重
w = conv.weight.data
# 2、
min_w, max_w = calcu_max_and_min(w, None, None, True)
# 3、
scale_w, zero_point_w = calcu_scale_and_zeropoint(min_w, max_w, 8, True)
# 4、
q_w = quantize_tensor(w, scale_w, zero_point_w,8,False, True)
# 5、
dq_w = dequantize_tensor(q_w, scale_w, zero_point_w, True)

print("原始：", w[0][0][0][:5])
print("量化：", q_w[0][0][0][:5])
print("反量化：", dq_w[0][0][0][:5])

原始： tensor([ 0.1693, -0.3146,  0.0790])
量化： tensor([200.0798,   0.0000, 162.6877])
反量化： tensor([ 0.1693, -0.3140,  0.0790])


#### 3、 进行对输出的量化

In [12]:
# 1
min_res, max_res = calcu_max_and_min(res, None, None, False)
# 2
scale_res, zero_point_res = calcu_scale_and_zeropoint(min_res, max_res, 8, False)
# 3
q_res = quantize_tensor(res, scale_res, zero_point_res).int()
# 4
dq_res = dequantize_tensor(q_res, scale_res, zero_point_res)

print("原始：", res[0][0][0][:5])
print("量化：", q_res[0][0][0][:5])
print("反量化：", dq_res[0][0][0][:5])

原始： tensor([ 0.8252,  0.2801,  0.6957, -1.0605,  1.0631], grad_fn=<SliceBackward>)
量化： tensor([162, 139, 157,  83, 172], dtype=torch.int32)
反量化： tensor([ 0.8155,  0.2638,  0.6955, -1.0793,  1.0553])


### 2.5 卷积中权值更新为量化权值

In [13]:
conv.weight.data = q_w

### 2.6 进行推理
<img src="https://www.zhihu.com/equation?tex=S_a+%28q_a-Z_a%29%3D%5Csum_%7Bi%7D%5EN+S_w%28q_w-Z_w%29S_x%28q_x-Z_x%29%2BS_b%28q_b-Z_b%29+%5Ctag%7B2%7D+" alt="[公式]" style="zoom:80%;" />
<img src="https://www.zhihu.com/equation?tex=q_a%3D%5Cfrac%7BS_w+S_x%7D%7BS_a%7D%5Csum_%7Bi%7D%5EN+%28q_w-Z_w%29%28q_x-Z_x%29%2B%5Cfrac%7BS_b%7D%7BS_a%7D%28q_b-Z_b%29%2BZ_a+%5Ctag%7B3%7D+" alt="[公式]" style="zoom:80%;" />

#### 1、量化输入

In [14]:
x_1 = q_a - zero_point_a

#### 2、进行卷积
> 注意：
> 1. 这里的输入已经模拟量化了
> 2. 卷积中的权重在上面某步中被模拟量化了（偏置被忽略了）

In [15]:
y_2 = conv(x_1)

#### 3、计算 M

![[公式]](https://www.zhihu.com/equation?tex=M%3D%5Cfrac%7BS_1+S_2%7D%7BS_3%7D)

注意：

> 1. M的计算公式根据逐层量化和逐通道量化方式的不同而不同，这里展示的是逐通道量化的公式

In [16]:
M = [scale_w[i] * scale_a / scale_res for i in range(len(scale_w))]
len(M)

40

#### 4、M 与 量化卷积乘积


In [17]:
for i in range(len(M)):
    y_2[:,i,:,:] = y_2[:,i,:,:] * M[i]

#### 5、加上 输出的zero_point
这里得到的是量化的输出结果

In [18]:
z_3 = y_2 +zero_point_res
    
z_3[0][0][0][:5]

tensor([220.0485, 148.4445, 217.4412, 156.8067, 255.8525],
       grad_fn=<SliceBackward>)

#### 6、反量化

In [19]:
dz_4 = dequantize_tensor(z_3, scale_res, zero_point_res)
dz_4[0][0][0][:5]

tensor([2.2077, 0.4903, 2.1452, 0.6909, 3.0664], grad_fn=<SliceBackward>)

#### 7、与原始结果对比

In [20]:
print("原始：", res[0][0][0][:5])
print("量化：", q_res[0][0][0][:5])
print("反量化：", dq_res[0][0][0][:5]) 

原始： tensor([ 0.8252,  0.2801,  0.6957, -1.0605,  1.0631], grad_fn=<SliceBackward>)
量化： tensor([162, 139, 157,  83, 172], dtype=torch.int32)
反量化： tensor([ 0.8155,  0.2638,  0.6955, -1.0793,  1.0553])


## 3、卷积中量化（封装）

### 3.1 定义一个卷积

In [21]:
conv = torch.nn.Conv2d(1, 40, 3, 1)

### 3.2 定义输入 a 并且输出指定范围数据

In [22]:
a = torch.randn((64, 1, 28, 28))
a[0][0][0][:5]

tensor([-0.7617,  0.7561, -0.7430, -0.0470, -0.3359])

### 3.3 卷积运算并且输出指定范围数据

In [23]:
a_out = conv(a)
a_out[0][0][0][:5]

tensor([-0.3136,  0.5206, -0.1667, -0.5224,  0.4417], grad_fn=<SliceBackward>)

### 3.4 定义量化卷积层

In [24]:
qconv1 = QConv2d(conv, has_qin=True, has_qout=True, num_bits=8)

### 3.5 计算S 和 z，进行中间参数的量化，并保存

In [25]:
y = qconv1(a)
qconv1.freeze()

### 3.6 量化输入

In [26]:
# 量化输入
qa = qconv1.q_in.quantize_tensor(a).int().float()
print(f"量化输入，展示部分数据：{qa[0][0][0][:5]}")

# 反量化输入
# 这部分没必要，只是为了给你们展示看效果
qaa = qconv1.q_in.dequantize_tensor(qa)
print(f"\n反量化输入，展示部分数据：\n\n{qaa[0][0][0][:5]}\n")
print(f"原始数据对比：\n\n{a[0][0][0][:5]}")

量化输入，展示部分数据：tensor([ 97., 144.,  98., 119., 110.])

反量化输入，展示部分数据：

tensor([-0.7770,  0.7446, -0.7446, -0.0648, -0.3561])

原始数据对比：

tensor([-0.7617,  0.7561, -0.7430, -0.0470, -0.3359])


### 3.7 使用量化输入进行推理

In [27]:
qa_out = qconv1.quantize_inference(qa)
print(f"推理输出，展示部分数据：{qa_out[0][0][0][:5]}")

qaa_out = qconv1.q_out.dequantize_tensor(qa_out)
print(f"\n反量化推理输出，展示部分数据：\n\n{qaa_out[0][0][0][:5]}\n")
print(f"原始推理结果对比：\n\n{a_out[0][0][0][:5]}")

推理输出，展示部分数据：tensor([120., 155., 126., 112., 151.], grad_fn=<SliceBackward>)

反量化推理输出，展示部分数据：

tensor([-0.3205,  0.5423, -0.1726, -0.5177,  0.4437], grad_fn=<SliceBackward>)

原始推理结果对比：

tensor([-0.3136,  0.5206, -0.1667, -0.5224,  0.4417], grad_fn=<SliceBackward>)


## 4、无法量化的缘故·教程

### 4.1 定义卷积

In [33]:
conv = torch.nn.Conv2d(1, 40, 3, 1)

### 4.2 定义输入并查看类型

In [34]:
a = torch.randn((64, 1, 28, 28))

print(f"shape: {a.shape} \ndtype: {a.dtype}")

shape: torch.Size([64, 1, 28, 28]) 
dtype: torch.float32


### 4.3 卷积运算（正常输出）

In [35]:
a_out = conv(a)

print(f"shape: {a_out.shape} \ndtype: {a_out.dtype}")

shape: torch.Size([64, 40, 26, 26]) 
dtype: torch.float32


### 4.4 改变输入类型

In [36]:
b = a.to(torch.uint8)

b.dtype

torch.uint8

### 4.5 卷积运算（错误输出）

In [37]:
b_out = conv(b)

RuntimeError: expected scalar type Byte but found Float

### 4.6 pytorch 目前无法量化的缘故（猜测）
- https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
- https://www.tensorflow.org/api_docs/python/tf/nn/conv2d#args

for pytorch:
> This module supports TensorFloat32.

for tensorflow:
> A Tensor. Must be one of the following types: half, bfloat16, float32, float64.